In [1]:
import os
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'

### GCN => GAT 

- GCN：Graph Conv Networks, GAT：Graph Attention Networks
- 都是用来学习 target node 的 representation
- gcn
    - $d_v$：表示节点的degree（即有多少条边连接），注意包含自己；
$$
\begin{align}
\mathbf{h}_{\mathcal{N}(v)} &= \sum_{u \in \mathcal{N}(v)} w_{u,v} \mathbf{h}_u \\
&= \sum_{u \in \mathcal{N}(v)} \sqrt{\frac{1}{d_v}} \sqrt{\frac{1}{d_u}} \mathbf{h}_u \\
&= \sqrt{\frac{1}{d_v}} \sum_{u \in \mathcal{N}(v)} \sqrt{\frac{1}{d_u}} \mathbf{h}_u
\end{align}
$$

- gat
    - https://www.youtube.com/watch?v=SnRfBfXwLuY

$$
\begin{align}
\mathbf{h}_{\mathcal{N}(v)} &= \sum_{u \in \mathcal{N}(v)} \underbrace{\text{softmax}_u \left( a(\mathbf{h}_u, \mathbf{h}_v) \right)}_{\alpha_{u,v}} \mathbf{h}_u \\
\alpha_{u,v} &= \frac{\exp\left(a(\mathbf{h}_u, \mathbf{h}_v)\right)}{\sum_{k \in \mathcal{N}(v)} \exp\left(a(\mathbf{h}_k, \mathbf{h}_v)\right)}
\end{align}
$$

### GAT

https://github.com/AntonioLonga/PytorchGeometricTutorial/blob/main/Tutorial3/Tutorial3.pdf
- https://github.com/AntonioLonga/PytorchGeometricTutorial/blob/main/Tutorial3/Tutorial3.ipynb
- input: a set of node features, $\mathbf{h} = \{ \bar{h}_1, \bar{h}_2, \dots, \bar{h}_n \}, \quad \bar{h}_i \in \mathbb{R}^{F}$
- output: a set of node features, $\mathbf{h'} = \{ \bar{h'}_1, \bar{h'}_2, \dots, \bar{h'}_n \}, \quad \bar{h'}_i \in \mathbb{R}^{F'}$
- GAT
    - by a parameterized linear transformation to every node
        - $\mathbf W\cdot \bar h_i, \mathbf W\in \mathbf R^{F'\times F}$
    - self attention:
        - $a: \mathbf R^{F'}\times \mathbf R^{F'} \rightarrow R$
        - $e_{ij}=a(\mathbf W\cdot \bar h_i,\mathbf W\cdot \bar h_j)$：specify the Importance of node j's features  to node i
    - normalization
        - $\alpha_{ij}=\text{softmax}(e_{ij})=\frac{\exp(e_{ij})}{\sum_{k\in N(i)}\exp(e_{ik})}$

#### 认识 Cora 数据集

- https://github.com/kimiyoung/planetoid
- Cora 是一个广泛使用的引用网络数据集，主要用于文献分类任务。它包含以下几个方面的信息：
    - 节点（Nodes）：每个节点代表一篇学术论文。
        - 特征（Features）：每个节点有一个高维的特征向量，通常是基于论文的词袋模型（Bag-of-Words）生成的。
    - 边（Edges）：边表示论文之间的引用关系，即一篇论文引用了另一篇论文。
    - 标签（Labels）：每篇论文被分配一个类别标签，表示其所属的研究领域，如机器学习、数据库、信息检索等。
- 加载Cora数据集后，您可以进行以下操作：
    - 节点分类：
        - 使用图神经网络对论文进行分类，预测未标记论文的类别。
    - 链接预测：预测论文之间是否存在引用关系。
    - 图嵌入：学习节点的低维表示，以便在下游任务中使用。

In [2]:
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T

import matplotlib.pyplot as plt

name_data = 'Cora'
dataset = Planetoid(root= './data/' + name_data, name = name_data)
dataset.transform = T.NormalizeFeatures()

Processing...
Done!


In [16]:
dataset.num_classes, dataset.num_node_features, dataset.num_edge_features, 

(7, 1433, 0)

In [21]:
# 边的属性如果有的话，叫 edge_attr??
dataset[0]

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

### coding

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
class GATLayer(nn.Module):
    """
    Simple PyTorch Implementation of the Graph Attention layer.
    """
    def __init__(self):
        super(GATLayer, self).__init__()
      
    def forward(self, input, adj):
        print("")

In [5]:
in_features = 5
out_features = 2
nb_nodes = 3

X = torch.rand(nb_nodes, in_features) 
W = nn.Parameter(torch.zeros(size=(in_features, out_features))) #xavier paramiter inizializator
nn.init.xavier_uniform_(W.data, gain=1.414)

X.shape, W.shape

(torch.Size([3, 5]), torch.Size([5, 2]))

In [6]:
h = torch.mm(X, W)
N = h.shape[0]
h.shape

torch.Size([3, 2])

In [7]:
h.repeat(1, N).view(N*N, -1)

tensor([[-0.1179, -0.3074],
        [-0.1179, -0.3074],
        [-0.1179, -0.3074],
        [ 1.5600,  0.5294],
        [ 1.5600,  0.5294],
        [ 1.5600,  0.5294],
        [ 1.2360,  0.6803],
        [ 1.2360,  0.6803],
        [ 1.2360,  0.6803]], grad_fn=<ViewBackward0>)

In [8]:
h.repeat(N, 1)

tensor([[-0.1179, -0.3074],
        [ 1.5600,  0.5294],
        [ 1.2360,  0.6803],
        [-0.1179, -0.3074],
        [ 1.5600,  0.5294],
        [ 1.2360,  0.6803],
        [-0.1179, -0.3074],
        [ 1.5600,  0.5294],
        [ 1.2360,  0.6803]], grad_fn=<RepeatBackward0>)

In [9]:
torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)], dim=1)

tensor([[-0.1179, -0.3074, -0.1179, -0.3074],
        [-0.1179, -0.3074,  1.5600,  0.5294],
        [-0.1179, -0.3074,  1.2360,  0.6803],
        [ 1.5600,  0.5294, -0.1179, -0.3074],
        [ 1.5600,  0.5294,  1.5600,  0.5294],
        [ 1.5600,  0.5294,  1.2360,  0.6803],
        [ 1.2360,  0.6803, -0.1179, -0.3074],
        [ 1.2360,  0.6803,  1.5600,  0.5294],
        [ 1.2360,  0.6803,  1.2360,  0.6803]], grad_fn=<CatBackward0>)

In [10]:
torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)], dim=1).view(N, -1, 2 * out_features)

tensor([[[-0.1179, -0.3074, -0.1179, -0.3074],
         [-0.1179, -0.3074,  1.5600,  0.5294],
         [-0.1179, -0.3074,  1.2360,  0.6803]],

        [[ 1.5600,  0.5294, -0.1179, -0.3074],
         [ 1.5600,  0.5294,  1.5600,  0.5294],
         [ 1.5600,  0.5294,  1.2360,  0.6803]],

        [[ 1.2360,  0.6803, -0.1179, -0.3074],
         [ 1.2360,  0.6803,  1.5600,  0.5294],
         [ 1.2360,  0.6803,  1.2360,  0.6803]]], grad_fn=<ViewBackward0>)

In [11]:
a_input = torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)], dim=1).view(N, -1, 2 * out_features)


In [12]:
leakyrelu = nn.LeakyReLU(0.2)  # LeakyReLU


In [13]:
a = nn.Parameter(torch.zeros(size=(2*out_features, 1))) #xavier paramiter inizializator
nn.init.xavier_uniform_(a.data, gain=1.414)
print(a.shape)

torch.Size([4, 1])


In [14]:
print(a_input.shape,a.shape)
print("")
print(torch.matmul(a_input,a).shape)
print("")
print(torch.matmul(a_input,a).squeeze(2).shape)

torch.Size([3, 3, 4]) torch.Size([4, 1])

torch.Size([3, 3, 1])

torch.Size([3, 3])
